In [2]:
import numpy as np
import stim
from tqdm import tqdm
from quits.qldpc_code import *
from quits.circuit import get_qldpc_mem_circuit
from quits.decoder import sliding_window_bposd_circuit_mem
from quits.simulation import get_stim_mem_result

In [23]:
from quits.ldpc_utility import (
    generate_ldpc,
    optimize_ldpc,
    has_duplicate_edges,
    binary_matrix_rank,
)

# Define LDPC parameters
n = 12  # Number of variable nodes
dv = 3  # Variable node degree
dc = 4  # Check node degree (m = n * dv / dc should be an integer)

# Generate an initial parity-check matrix
H = generate_ldpc(n, dv, dc)
print("Original parity-check matrix H:")
print(H)

# Perform initial optimization
rounds = 100
print(f"\nOptimizing for {rounds} rounds...")
H = optimize_ldpc(H, rounds, max_depth=10)

# Continue optimizing until constraints are satisfied
while has_duplicate_edges(H) or binary_matrix_rank(H) < H.shape[0]:

    # Additional optimization rounds
    rounds = 10
    print(f"\nOptimizing for another {rounds} rounds...")
    H = optimize_ldpc(H, rounds, max_depth=10)

# Display the final optimized parity-check matrix
print(f"\nOptimized parity-check matrix H (rank {binary_matrix_rank(H)}):")
print(H)

# Further optimization can be performed if needed.

Original parity-check matrix H:
[[0 1 0 0 0 1 1 0 0 0 0 1]
 [0 0 0 0 1 0 0 1 0 0 1 1]
 [0 0 1 1 0 0 0 0 1 1 0 0]
 [3 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 1 0 0 0 2 0 1 0]
 [0 0 0 0 1 1 0 1 0 1 0 0]
 [0 0 2 0 0 0 0 0 0 1 1 0]
 [0 0 0 2 0 1 0 0 0 0 0 1]
 [0 1 0 0 0 0 2 1 0 0 0 0]]

Optimizing for 100 rounds...
(2, np.int64(1)),(2, np.int64(1)) -> (4, np.int64(5)),(4, np.int64(5))
(4, np.int64(1)),(2, np.int64(3)) -> (4, np.int64(1)),(2, np.int64(1))
(4, np.int64(2)),(4, np.int64(5)) -> (4, np.int64(4)),(4, np.int64(2))
(2, np.int64(1)),(4, np.int64(1)) -> (4, np.int64(2)),(4, np.int64(2))
(4, np.int64(4)),(4, np.int64(1)) -> (4, np.int64(3)),(4, np.int64(2))
(4, np.int64(4)),(2, np.int64(1)) -> (4, np.int64(1)),(4, np.int64(2))
(4, np.int64(4)),(4, np.int64(2)) -> (4, np.int64(1)),(6, np.int64(12))
(4, np.int64(3)),(4, np.int64(1)) -> (4, np.int64(2)),(4, np.int64(1))
(2, np.int64(1)),(4, np.int64(2)) -> (2, np.int64(1)),(4, np.int64(1))
(4, np.int64(2)),(4, np.int64(1)) -> (6, np.int64(18))

In [24]:
from ldpc.code_util import compute_code_parameters

n, k, d_estimate = compute_code_parameters(H, timeout_seconds=1)
print(f"Code parameters: [n = {n}, k = {k}, d <= {d_estimate}]")

Code parameters: [n = 12, k = 3, d <= 4]


import 

In [17]:
code = HgpCode(H, H)
code.build_graph(seed=42)

In [18]:
num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values()))
print("# data qubits: ", num_data, " # logical qubits: ", num_logical)
print("# z-check qubits: ", num_zcheck, " # x-check qubits: ", num_xcheck)
print("# layers of entangling gates: ", depth)

# data qubits:  125  # logical qubits:  25
# z-check qubits:  50  # x-check qubits:  50
# layers of entangling gates:  12


In [21]:
p = 1e-3  # physical error rate
num_rounds = 5  # number of rounds (T-1)
basis = "Z"  # 'Z' or 'X'

circuit = stim.Circuit(
    get_qldpc_mem_circuit(
        code, p, p, p, p, num_rounds, basis=basis, noisy_init=True, noisy_meas=False
    )
)

## Load QLDPC codes and circuits

### Hypergraph product (HGP) code

Load the parity check matrix of the classical LDPC code stored in '../parity_check_matrices' directory.  
The parity check matrix `h` is found from `quits.ldpc_utility.generate_ldpc`, following the method in A. Grospellier, *Constant time decoding of quantum expander codes and application to fault-tolerant quantum computation* (2019).

In [2]:
# Load the parity check matrix of the classical code that is used to construct the hgp code
h = np.loadtxt("../parity_check_matrices/n=12_dv=3_dc=4_dist=6.txt", dtype=int)
print("Shape of classical code's parity check matrix: ", h.shape)

Shape of classical code's parity check matrix:  (9, 12)


In [3]:
code = HgpCode(h, h)  # Define the HgpCode object
code.build_graph(seed=22)  # Build the Tanner graph and assign directions to its edges.
# For this specific h, seed=22 gives a circuit with entangling depth 8.
num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values()))
print("# data qubits: ", num_data, " # logical qubits: ", num_logical)
print("# z-check qubits: ", num_zcheck, " # x-check qubits: ", num_xcheck)
print("# layers of entangling gates: ", depth)

# data qubits:  225  # logical qubits:  9
# z-check qubits:  108  # x-check qubits:  108
# layers of entangling gates:  8


In [4]:
p = 1e-3  # physical error rate
num_rounds = 15  # number of rounds (T-1)
basis = "Z"  # 'Z' or 'X'

circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
# print(circuit)

### Quasi-cyclic lifted product (QLP) code

The base matrix `b` is brought from Q. Xu et al., *Nat. Phys.* 20, 1084 (2024). Each entry represents the exponent of the monomial. 

In [5]:
lift_size = 16
b = np.array(
    [
        [0, 0, 0, 0, 0],  # [e, e, e, e, e]
        [0, 2, 4, 7, 11],  # [e, x^2, x^4, x^7, x^11]
        [0, 3, 10, 14, 15],  # [e, x^3, x^10, x^14, x^15]
    ]
)

In [6]:
code = QlpCode(b, b, lift_size)  # Define the QlpCode object
code.build_graph(seed=1)  # Build the Tanner graph and assign directions to its edges.

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values()))
print("# data qubits: ", num_data, " # logical qubits: ", num_logical)
print("# z-check qubits: ", num_zcheck, " # x-check qubits: ", num_xcheck)
print("# layers of entangling gates: ", depth)

# data qubits:  544  # logical qubits:  80
# z-check qubits:  240  # x-check qubits:  240
# layers of entangling gates:  12


In [7]:
p = 1e-3  # physical error rate
num_rounds = 15  # number of rounds (T-1)
basis = "Z"  # 'Z' or 'X'

circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
# print(circuit)

### Balanced product cyclic (BPC) code

This code is introduced in R. Tiew & N. P. Breuckmann, arXiv:2411.03302. 

In [8]:
lift_size, factor = 15, 3
p1 = [0, 1, 5]  # e + x + x^5
p2 = [0, 2, 7]  # e + x^2 + x^7

In [9]:
code = BpcCode(p1, p2, lift_size, factor)  # Define the BpcCode object
code.build_graph(seed=1)  # Build the Tanner graph and assign directions to its edges.

num_zcheck, num_data = code.hz.shape
num_xcheck, num_data = code.hx.shape
num_logical = code.lz.shape[0]
depth = sum(list(code.num_colors.values()))
print("# data qubits: ", num_data, " # logical qubits: ", num_logical)
print("# z-check qubits: ", num_zcheck, " # x-check qubits: ", num_xcheck)
print("# layers of entangling gates: ", depth)

# data qubits:  90  # logical qubits:  8
# z-check qubits:  45  # x-check qubits:  45
# layers of entangling gates:  8


In [10]:
p = 2e-3  # physical error rate
num_rounds = 15  # number of rounds (T-1)
basis = "Z"  # 'Z' or 'X'

circuit = stim.Circuit(get_qldpc_mem_circuit(code, p, p, p, p, num_rounds, basis=basis))
# print(circuit)

# Simulate the QEC performance

In [11]:
num_trials = 100
## Simulate the circuit using Stim.
detection_events, observable_flips = get_stim_mem_result(
    circuit, num_trials, seed=1
)  # simulate the circuit using Stim

W, F = 5, 3  # sliding window parameters
max_iter, osd_order = 20, 10  # BP-OSD decoder parameters

# Perform decoding of the detection_events generated from simulating the circuit.
# Returns the logical observable flip predicted from decoding.
logical_pred = sliding_window_bposd_circuit_mem(
    detection_events,
    circuit,
    code.hz,
    code.lz,
    W,
    F,
    max_iter=max_iter,
    osd_order=osd_order,
    tqdm_on=True,
)

# Logical error is recorded whenever logical_pred does not match observable_flips for any logical qubit at any round
pL = np.sum((observable_flips - logical_pred).any(axis=1)) / num_trials
lfr = 1 - (1 - pL) ** (1 / num_rounds)
print("p: %.7f, LFR: %.7f" % (p, lfr))

100%|██████████| 100/100 [00:47<00:00,  2.12it/s]

p: 0.0020000, LFR: 0.0006698


## Example of using a customized inner decoder for sliding-window decoding

In [14]:
# use the BP-OSD decoder from the LDPC package (https://software.roffe.eu/ldpc/) as an example
from quits.decoder import sliding_window_bposd_circuit_mem, sliding_window_circuit_mem
from ldpc.bposd_decoder import BpOsdDecoder

"""
Note that it is allowed to use a different decoder (Decoder2) for the last decoding window 
than the one used for the other windows (Decoder1).
This feature is useful when you want to: 
   i.e. running BP decoder for all other windows but use BPOSD for the last window to make sure the correction match the syndrome
   
The name of the decoder class is the BpOsdDecoder
The method of this class that takes in a syndrome and returns a correction is "decode"
The parameter of BpOsdDecoder that stores the individual error rate for different error mechanisms is "channel_probs"
An instance of this class is initialized with a parity check matirx, "channel_probs", and the other parameters listed below
"""
# Other paramters that defines the decoders
dict1 = {
    "bp_method": "product_sum",
    "max_iter": max_iter,
    "schedule": "serial",
    "osd_method": "osd_cs",
    "osd_order": osd_order,
}
dict2 = {
    "bp_method": "product_sum",
    "max_iter": max_iter,
    "schedule": "serial",
    "osd_method": "osd_cs",
    "osd_order": osd_order,
}
# Returns the logical observable flip predicted from decoding.
logical_pred = sliding_window_circuit_mem(
    detection_events,
    circuit,
    code.hz,
    code.lz,
    W,
    F,
    BpOsdDecoder,
    BpOsdDecoder,
    dict1,
    dict2,
    "channel_probs",
    "channel_probs",
    "decode",
    "decode",
    tqdm_on=True,
)